# First NGSolve example

Let us solve the Poisson problem: 

$$\text{find: } u \in H_{0,D}^1 \quad \int_\Omega \nabla u \nabla v = \int_\Omega f v \quad \forall v \in H_{0,D}^1$$

## Quick steps to solution:

#### 1. Import NGSolve and Netgen Python modules:

In [1]:
import netgen.gui
%gui tk
from ngsolve import *
from netgen.geom2d import unit_square

#### 2. Generate an unstructured mesh

In [2]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
mesh.nv, mesh.ne   # number of vertices & elements 

(37, 52)

Here we prescribed a maximal mesh-size of 0.2 using the `maxh` flag. The mesh can be viewed by clicking on the `Mesh` tab in the Netgen window. 

#### 3. Declare a finite element space:

In [3]:
fes = H1(mesh, order=2, dirichlet="bottom|right")
fes.ndof  # number of unknowns in this space

125

Python's help system displays further documentation.

In [4]:
help(fes)

Help on FESpace in module ngsolve.comp object:

class FESpace(pybind11_builtins.pybind11_object_56)
 |  Finite Element Space
 |  
 |  Provides the functionality for finite element calculations. Use
 |  the finite element space generator functions to construct the space,
 |  this can sometimes provide additional functionality (e.g HCurl).
 |  When createing a FESpace with a generator function the set parameters
 |  are passed to the FESpace constructor with and the type parameter is
 |  set. If the space has additional functionality it is added to the space.
 |  
 |  Available generator functions
 |  
 |  H1
 |  HCurl
 |  HDiv
 |  L2
 |  FacetFESpace
 |  HDivDiv
 |  
 |  2 __init__ overloads:
 |    1) To create a registered FESpace
 |    2) To create a compound FESpace from multiple created FESpaces
 |  
 |  1)
 |  
 |  Parameters
 |  
 |  type : string
 |    Type of the finite element space. This parameter is automatically
 |    set if the space is constructed with a generator function

#### 4. Declare test function, trial function, and grid function 

Test and trial function are symbolic objects that help you construct bilinear forms (and have no space to hold solutions). Grid functions represent functions in the finite element space and contains memory to hold coefficient vectors.

In [5]:
u = fes.TrialFunction()  # symbolic object
v = fes.TestFunction()   # symbolic object
gfu = GridFunction(fes)  # solution 

#### 5. Define and assemble linear and bilinear forms:

In [6]:
a = BilinearForm(fes, symmetric=True)
a += SymbolicBFI(grad(u)*grad(v))
a.Assemble()

f = LinearForm(fes)
f += SymbolicLFI(x*v)
f.Assemble()

You can examine the linear system in more detail:

In [7]:
print(f.vec)

 0.000333333
 0.0103012
 0.00633333
 0.000333333
 0.00387671
 0.006844
 0.00946082
 0.0131276
 0.00932238
 0.0231198
 0.0160023
 0.024313
 0.0168221
 0.0111949
 0.00812513
 0.00410081
 0.00276968
 0.000493909
 0.00137444
 0.00297784
 0.0117697
 0.0200805
 0.0283719
 0.0282328
 0.0279711
 0.0134645
 0.00752298
 0.0155354
 0.0257406
 0.0228613
 0.0214477
 0.00993661
 0.0137531
 0.0156134
 0.0216362
 0.0237827
 0.0210528
 -6.66667e-05
 -3.33333e-05
 -0.000687449
 -0.000587035
 -0.00126541
 -0.0008
 -0.000766667
 -6.66667e-05
 -3.33333e-05
 -0.000228362
 -0.000220805
 -0.00042906
 -0.00033644
 -0.000514805
 -0.000625037
 -0.000462964
 -0.000842943
 -0.000719098
 -0.000956807
 -0.00116943
 -0.000587035
 -0.00113898
 -0.000776995
 -0.00136243
 -0.00146798
 -0.00149124
 -0.000668081
 -0.00130141
 -0.00120446
 -0.00164251
 -0.00152774
 -0.00132933
 -0.00049277
 -0.00133031
 -0.000399239
 -0.00100556
 -0.000902792
 -0.000250598
 -0.00060796
 -0.000769314
 -0.000227209
 -0.000456957
 -2.46955e-0

In [8]:
print(a.mat)

Row 0:   0: 1
Row 1:   1: 0.870166
Row 2:   2: 1
Row 3:   3: 1
Row 4:   0: -0.5   4: 1.89594
Row 5:   4: -0.328601   5: 1.75373
Row 6:   5: -0.252529   6: 1.74397
Row 7:   1: -0.355392   6: -0.246158   7: 1.75039
Row 8:   1: -0.315451   8: 2.11144
Row 9:   8: -0.417618   9: 1.74594
Row 10:   9: -0.35985   10: 1.7563
Row 11:   2: -0.5   10: -0.267891   11: 1.8234
Row 12:   2: -0.5   11: -0.157475   12: 1.9414
Row 13:   12: -0.279511   13: 1.73971
Row 14:   13: -0.347754   14: 1.75594
Row 15:   3: -0.5   14: -0.389763   15: 1.87732
Row 16:   3: -0.5   15: -0.203686   16: 1.80816
Row 17:   16: -0.436369   17: 2.02265
Row 18:   17: -0.424487   18: 1.88532
Row 19:   0: -0.5   4: -0.19922   18: -0.505055   19: 1.8246
Row 20:   4: -0.868115   5: -0.442585   19: -0.376723   20: 3.59693
Row 21:   6: -0.713275   7: -0.495693   21: 3.52299
Row 22:   1: -0.199323   7: -0.653151   8: -1.37837   9: -0.102645   21: -0.73776   22: 3.94083
Row 23:   9: -0.484277   10: -0.45587   23: 3.57987
Row 24:   1

#### 6. Solve the system:

In [9]:
gfu.vec.data = \
    a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec
Draw(gfu)

The Dirichlet boundary condition constrains some degrees of freedom. The argument `fes.FreeDofs()` indicates that only the remaining "free" degrees of freedom should participate in the linear solve.

You can examine the coefficient vector of solution if needed:

In [10]:
print(gfu.vec)

       0
       0
       0
 0.0924091
       0
       0
       0
       0
       0
       0
       0
       0
 0.0579249
 0.0863395
 0.095412
 0.0945061
 0.0887126
 0.0780983
 0.0595882
 0.0330527
 0.0347201
 0.0306088
 0.0226997
 0.0485255
 0.0702498
 0.0920839
 0.0799272
 0.0334133
 0.0451654
 0.0475731
 0.089964
 0.0608074
 0.0521306
 0.0604622
 0.0816294
 0.0755969
 0.067892
       0
 -0.00580903
       0
       0
 0.0247054
       0
 -0.0349026
 0.00339392
 -0.00739309
       0
 -0.0100181
 -0.00505004
       0
 -0.0147034
 -0.00938597
       0
 -0.017922
 -0.0121489
 -0.00529069
 -0.0292704
       0
 -0.0183813
       0
 -0.0461399
 -0.0215322
 -0.05256
       0
 -0.0378931
 -0.0206654
 -0.0330051
 -0.0606934
 -0.0276814
 -0.024247
 -0.00671107
 -0.0145564
 -0.0107763
 -0.00927653
 -0.00544311
 -0.00971665
 -0.00812781
 -0.00479834
 -0.00711323
 -0.00781592
 0.00251074
 -0.00483204
 -0.00805012
 0.00347202
 -0.0083864
 -0.0049253
 0.00455957
 0.00401028
 -0.00241395
 -0.00346299
